# Collect shapes and generate relations

In this folder, we are building a dataset of geospatial shapes that have certain perscribed
relations, like "polygon intersection" or "point-on-line". 
The actual entities and their specific attributes don't matter. 

All shapes will be pulled from OpenStreetMap using the `osmnx` package. 
We will not be encoding things in lon/lat space.
Instead we will focus on rectangular subsets of say 10km x 10 km. 
The mapping between lon/lat and local x/y will be done based on a local
transverse Mercator projection.


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx
import shapely
import pyproj

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter


In [ ]:
from geo_relations import OSMShapeCollector
from geo_relations import RelationGenerator
from geo_relations import draw_shape

## Setup

In [ ]:
# Define a lon/lat center and a box size from which to pull shapes.
center_lat, center_lon = 42.631024, -70.993787 # Boxford MA
extent = 20000.0 # meters; this represents both width and height

In [ ]:
# Get a set of Point, LineString, and Polygon objects from our sample box.
collector = OSMShapeCollector(center_lon, center_lat, extent)
shapes = collector.collect(['points', 'linestrings', 'polygons'])
shapes['type'].value_counts()

In [ ]:
# Get some tiled polygons too. These will be used to generate 
# "polygon-borders-polygon" relations.
collector = OSMShapeCollector(center_lon, center_lat, extent * 2)
tiles = collector.collect(['tiled-polygons'])
tiles['type'].value_counts()

In [ ]:
region_width = 100
region_height = 100
relation = 'point-in-polygon'
fodder = shapes
generator = RelationGenerator(fodder, bounds=[0, 0, region_width, region_height], scale=[0.1, 0.5], margin=20)


In [ ]:
# g0 = generator.pick_a_random('Polygon')
# g1 = generator.rescale(g0)
# g2 = generator.reposition(g1)
# g3 = generator.rotate(g2)

# fig = make_subplots(1, 1)
# draw_shape(g2, fig, color='red')
# draw_shape(g3, fig, color='purple')
# fig['layout']['width'] = 500
# fig['layout']['height'] = 500
# fig['layout']['xaxis1']['range'] = [0, 100]
# fig['layout']['yaxis1']['range'] = [0, 100]
# fig.show()

In [ ]:
n = 4
fig = make_subplots(1, n)

for icol in range(n):
    a, b = generator.generate(relation='polygon-borders-polygon', sense=True, max_attempts=100)
    if a is None:
        continue
    draw_shape(a, fig, irow=1, icol=icol+1, color='red')
    draw_shape(b, fig, irow=1, icol=icol+1, color='blue')
    fig['layout']['xaxis%d'%(icol+1)]['range'] = [0, 100]
    fig['layout']['yaxis%d'%(icol+1)]['range'] = [0, 100]
fig['layout']['width'] = 1200
fig['layout']['height'] = 400
fig.show()